# L5b: Flux Balance Analysis (FBA) and Linear Programming
In this lecture, we'll explore the structure of the [flux balance analysis (FBA) problem](https://pubmed.ncbi.nlm.nih.gov/20212490/). The key ideas of this lecture include:
* __Flux Balance Analysis (FBA)__ is a mathematical approach for analyzing the steady-state flow of carbon and energy through a metabolic network operating in some abstract volume, e.g., a cell, a test tube, or a logical compartment. The flow of material through the reaction network is called the _metabolic flux_. We estimate metabolic flux using [linear programming](https://en.wikipedia.org/wiki/Linear_programming) (not the only approach, but the one that we will start with). 
* __Metabolic flux__, the rate of molecular flow in reaction pathways, is crucial for understanding cellular operations. It reveals how cells manage energy production, utilize nutrients, and respond to environmental changes, making it vital for studying diseases, drug development, and optimizing bioprocesses.
* __Linear programming__ is an optimization technique used to find the _best_ outcome in a mathematical model whose requirements are represented by linear relationships, e.g., the maximization (minimization) of a linear objective function subject to linear equality, inequality, and bounds constraints. [For a _deep dive_ into linear programming, see the ORIE 6300 course notes!](https://people.orie.cornell.edu/dpw/orie6300/)

Lecture notes can be downloaded: [here!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5b/docs/Notes.pdf)

## Motivation
[Flux balance analysis (FBA)](https://pubmed.ncbi.nlm.nih.gov/20212490/) enables researchers to _estimate_ metabolic fluxes and optimize cellular metabolism, providing insight into biological systems' operation without requiring extensive kinetic parameter information. FBA can operate in data-rich and data-poor situations.
* __Structure__. The flux balance analysis problem is a linear program composed of an objective (what we are trying to optimize), constraints (the rules of the world, in our case material balances), and bounds (limits on the decision variables). It returns the optimal values for the reaction rates in a system (metabolic fluxes).
* __Integration__. Flux balance analysis integrates measurement data (extracellular uptake, intracellular omics, etc) using a model of the global operation of a cell. FBA provides a _snapshot_ of the state of a system. While more data enhances realism, FBA has low data requirements.

Flux balance analysis (FBA) has some _perceived_ limitations.
* __Not unique__. FBA does not specify fluxes in a metabolic network uniquely, as regulatory mechanisms affecting enzyme kinetics and expression influence the chosen flux distribution, resulting in multiple potential flux solutions for an optimal state. This limitation is __major__ (and true).
* __Not dynamic__. FBA cannot model dynamic metabolic behavior due to its steady-state assumption, limiting its ability to capture temporal changes. However, [we can adapt FBA to be approximately dynamic](https://pmc.ncbi.nlm.nih.gov/articles/PMC1302231/), making this limitation minor.
* __No regulation__. FBA may conflict with experimental data, especially when regulatory loops are excluded. These discrepancies reveal the limitations of relying only on stoichiometric information without considering complex cellular regulation. This can be fixed with [regulatory flux balance analysis](https://pubmed.ncbi.nlm.nih.gov/11708855/). Gene expression is _easy(ish)_, but allosteric regulation (activity) is hard.

Example FBA publications:
* [Edwards JS, Ibarra RU, Palsson BO. In silico predictions of Escherichia coli metabolic capabilities are consistent with experimental data. Nat Biotechnol. 2001 Feb;19(2):125-30. doi: 10.1038/84379. PMID: 11175725.](https://pubmed.ncbi.nlm.nih.gov/11175725/)
* [Vilkhovoy M, Horvath N, Shih CH, Wayman JA, Calhoun K, Swartz J, Varner JD. Sequence-Specific Modeling of E. coli Cell-Free Protein Synthesis. ACS Synth Biol. 2018 Aug 17;7(8):1844-1857. doi: 10.1021/acssynbio.7b00465. Epub 2018 Jul 16. PMID: 29944340.](https://pubmed.ncbi.nlm.nih.gov/29944340/)
* [Tan ML, Jenkins-Johnston N, Huang S, Schutrum B, Vadhin S, Adhikari A, Williams RM, Zipfel WR, Lammerding J, Varner JD, Fischbach C. Endothelial cells metabolically regulate breast cancer invasion toward a microvessel. APL Bioeng. 2023 Dec 4;7(4):046116. doi: 10.1063/5.0171109. PMID: 38058993; PMCID: PMC10697723.](https://pubmed.ncbi.nlm.nih.gov/38058993/)

## Theory: Material Balances
Suppose we have [a system with abstract volume $V$](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5b/docs/figs/Fig-System-Schematic.pdf), e.g., the physical volume inside a single cell, the physical volume in a test tube, or the mass of cells in a reactor, etc. Inside this system, we have reaction set $\mathcal{R}$, metabolite set $\mathcal{M}$, and a stream set $\mathcal{S}$ that connects the system and the surroundings. A material balance equation for species $i\in\mathcal{M}$ in a system with volume $V$ has four terms:
$$
\begin{equation}
\text{Accumulation} = \text{Generation} + \text{Transport In} - \text{Transport Out}
\end{equation}
$$
The terms of the material balance are defined as:
* The __accumulation__ term is the rate of change of species $i$ in the system, the __generation__ term is the rate of production (consumption) of species $i$ by chemical reactions in the system, and the __transport__ terms describe the rate of physical (convection or passive diffusion) or logical transport of species $i$ into (from) the system.

Let's look at two material balances, namely, the dynamic species mole balance and the dynamic concentration balance equations.

### Dynamic species mole balances
The number of moles $n_{i}$ (unit: `mol`) in a system as a function of time is described by the _open species mole balance equation_:
$$
\begin{equation}
\sum_{s\in\mathcal{S}}d_{s}\dot{n}_{i,s} + \dot{n}_{G,i} = \frac{dn_{i}}{dt}
\qquad\forall{i}\in\mathcal{M}
\end{equation}
$$
where $\dot{n}_{i,s}$ is the mole flow rate of species $i$ in stream $s$ (units: `mol/time`),
$\dot{n}_{G,i}$ is the generation rate of species $i$ in the system 
(units: `mol/time`), and $dn_{i}/dt$ denotes is the rate of accumulation of species $i$ in the system (units: `mol/time`). The terms $d_{s}$ denote direction parameters: $d_{s}$ = `1` if stream $s\in\mathcal{S}$ enters the system, 
while $d_{s}$ = `-1` if stream $s\in\mathcal{S}$ exits the system. 

#### Generation
The species generation rate $\dot{n}_{G,i}$ can be written in terms of the open extent of reaction:
$$
\begin{equation}
\dot{n}_{G,i} = \sum_{r\in\mathcal{R}}\sigma_{ir}\dot{\epsilon}_{r}
\end{equation}
$$
where $\sigma_{ir}$ is the stoichiometric coefficient of species $i$ in reaction $r$, and $\dot{\epsilon}_{r}$ is the open extend of reaction $r$ (units: `mol/time`). Putting these ideas together, we can rewrite the _open species mole balance_ as:
$$
\begin{equation}
\sum_{s\in\mathcal{S}}d_{s}\dot{n}_{i,s} + \sum_{r\in\mathcal{R}}\sigma_{ir}\dot{\epsilon}_{r} = \frac{dn_{i}}{dt}\qquad\forall{i\in\mathcal{M}}
\end{equation}
$$

### Dynamic species concentration balances
When describing systems with chemical reactions, we write reaction rate expressions in terms of concentration, e.g., mole per unit volume basis. The number of moles $n_{i}$ (units: `mol`) of species $i$ in a system is described by an _open species mole balance equation_:
$$
\begin{equation}
\sum_{s\in\mathcal{S}}\nu_{s}\dot{n}_{i,s} + \dot{n}_{G,i} = \frac{dn_{i}}{dt}
\qquad\forall{i}\in\mathcal{M}
\end{equation}
$$
However, we can re-write the number of moles of species $i$ as $n_{i} = C_{i}V$ for $i\in\mathcal{M}$
where $C_{i}$ is the concentration of species $i$ (units: `mole per volume`), and $V$ (units: `volume`) is the volume of the system. The species mole balance can be rewritten in concentration units as:
$$
\begin{equation}
\sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s} + \dot{C}_{G,i}V = \frac{d}{dt}\left(C_{i}V\right)\qquad\forall{i}\in\mathcal{M}
\end{equation}
$$
where $\dot{V}_{s}$ denotes the volumetric flow rate for stream $s$ (units: `volume/time`), $C_{i,s}$ denotes the concentration of species $i$ in stream $s$ (units: `concentration`), and $\dot{C}_{G,i}$ is the rate of generation of species $i$ by chemical reaction (units: `concentration/time`). 

#### Generation
The generation terms for species $i$ in the concentration balance can be written as:
$$
\begin{equation}
\dot{C}_{G,i}V = \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V\qquad\forall{i}\in\mathcal{M}
\end{equation}
$$
where $\sigma_{ij}$ denotes the stoichiometric coefficient of species $i$ in reaction $j$ (units: `dimensionless`), 
and $\hat{v}_j $ denotes the rate of the jth chemical reaction per unit volume (units: `concentration/volume-time`), and $V$ denotes the volume of the system (units: `volume`). Putting these ideas together gives the _species concentration balance_:
$$
\begin{equation}
\sum_{s\in\mathcal{S}}d_{s}C_{i,s}\dot{V}_{s} + \sum_{j\in\mathcal{R}}\sigma_{ij}\hat{v}_{j}V = \frac{d}{dt}\left(C_{i}V\right)\qquad\forall{i\in\mathcal{M}}
\end{equation}
$$
Note: the transport terms are shown as physical terms, but we could also write logical flow terms.

## Theory: Linear Programming
Let $\mathcal{O}(\mathbf{x})$ denote a _linear function_ of the non-negative decision variables 
$\mathbf{x}\in\mathbb{R}^{n}$
whose values are constrained by a system of linear algebra equations and bounded. 
Then, the optimal decision $\mathbf{x}^{\star}\in\mathbb{R}^{n}$ is the solution of the _linear program_ (written in standard form):
$$
\begin{align*}
\max_{\mathbf{x}} &\quad \mathcal{O}(\mathbf{x}) = \sum_{i=1}^{n} c_{i}{x}_{i}\\
\text{subject to}&\quad\mathbf{A}\mathbf{x} \leq\mathbf{b}\\
\text{and} &\quad x_{i}\geq{0}\quad{i=1,2,\dots,n}
\end{align*}
$$
where $c_{i}\in\mathbb{R}$ are constant coefficients in the objective function, $\mathbf{A}\in\mathbb{R}^{m\times{n}}$ 
is an $m\times{n}$ constraint matrix and $\mathbf{b}\in\mathbb{R}^{m}$ is an $m\times{1}$ right-hand side vector. 
_Any linear program can be converted into this standard form_.  [Click me to see a schematic of the solution of this problem!](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5b/docs/figs/Fig-LinearProgramming-Schematic.pdf). 

* __How do we solve this__? There exist _very efficient_ solution methods for this type of problem; see [the simplex method of Danzig](https://en.wikipedia.org/wiki/Simplex_algorithm). In our case, we'll use [the `GLPK.jl` package](https://github.com/jump-dev/GLPK.jl), which is a wrapper around [the GNU Linear Programming Kit (GLPK) solver](https://www.gnu.org/software/glpk/).
* For a _much much deep dive_ into linear programming, [see the ORIE 6300 course notes](https://people.orie.cornell.edu/dpw/orie6300/). We will not dig into how the solver works or the differences between the different solution approaches. However, (time permitting) we may dig into some interesting things like [the dual](https://en.wikipedia.org/wiki/Dual_linear_program).

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 
* The `Include.jl` file also loads external packages, various functions that we will use in the exercise, and custom types to model the components of our problem. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Other packages are downloaded and then loaded.

In [8]:
include("Include.jl");

## Example: Toy Network Problem
Let's consider [a toy network example](https://github.com/varnerlab/CHEME-5450-Lectures-Spring-2025/blob/main/lectures/week-5/L5b/docs/figs/Fig-ToyNetwork-CBT_v2.pdf) to become familiar with the structure of the FBA linear programming problem. 

Suppose we have a test tube (physical volume), and a logical control volume in which all the chemistry takes place. Let's use flux balance analysis, in combination with species mole balances, to analyze this system, i.e., compute the optimal extent of reaction $\dot{\epsilon}_{r}$.
* __Key assumption__: Everything inside the _logical_ control volume operates at a steady state. However, the physical volume is _not at steady-state_. Thus, the system (physical + logical control volumes) is at a pseudo steady-state. This system is open; it can exchange material (energy) with the surroundings. 

The species in the problem will be $\mathcal{M} = \left\{A_{1},A_{2},B, P, c, x, y\right\}$ while the reactions will be $\mathcal{R} = \left\{r_{1},r_{2},r_{3}\right\}$.

#### Constraints
The steady-state species mole balances will be the constraints of the linear programming problem. The steady-state balance around component $i$ in the _logical control volume_ is given by:
$$
\begin{equation}
\sum_{s=1}^{\mathcal{S}}d_{s}\dot{n}_{is} + \sum_{j=1}^{\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j} = 0\qquad{i=1,2,\dots,\mathcal{M}}
\end{equation}
$$
Assuming a single logical input (`s = 1`) and exit (`s = 2`) stream, the steady-state open species mole balance is given by:
$$
\begin{equation}
\dot{n}_{i,2} = \dot{n}_{i,1} + \sum_{j=1}^{\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}\qquad{i=1,2,\dots,\mathcal{M}}
\end{equation}
$$
Thus, because $\dot{n}_{i,2}\geq{0}$, the linear programming problem will be subject to the mol balance constraints: 
$$
\begin{equation}
\sum_{j=1}^{\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}\geq{-\dot{n}_{i,1}}\qquad\qquad{i=1,2,\dots,\mathcal{M}}
\end{equation}
$$

#### Bounds constraints
Next, the open extent $\dot{\epsilon}_{j}$ terms (the unknowns  we are trying to estimate) are bounded from above and below: $\mathcal{L}_{j}\leq\dot{\epsilon}_{j}\leq\mathcal{U}_{j}$ by bounds constraints,
where the $\mathcal{L}_{j}$ and $\mathcal{U}_{j}$ denote the lower and upper bounds that $\dot{\epsilon}_{j}$ can take. 
* __What are bounds contraints__? Open extents $\dot{\epsilon}_{j}$ are just reaction rates times the volume. Thus, the lower and upper bounds describe the permissible range we expect the rate _could_ obtain.

Putting everything together gives the linear programming problem formulation for the open extent $\dot{\epsilon}_{j}$:
$$
\begin{align}
\max_{\dot{\epsilon}_{r}}\quad & \sum_{j=1}^{\mathcal{R}}c_{j}\dot{\epsilon}_{j} \\
\text{subject to}\quad & \sum_{j=1}^{\mathcal{R}}\sigma_{ij}\dot{\epsilon}_{j}\geq{-\dot{n}_{i,1}}\qquad\forall{i}\\
& \mathcal{L}_{j}\leq\dot{\epsilon}_{j}\leq\mathcal{U}_{j}\qquad{j=1,2\dots,\mathcal{R}}
\end{align}
$$

__Input streams__: First, let's specify the composition of the input stream $\dot{n}_{i,1}$. We'll store the in the `n_dot_in::Array{Float64,1}` vector. Each row in the `n_dot_in::Array{Float64,1}` vector corresponds to a particular species in $i\in\mathcal{M}$.

In [11]:
n_dot_in = [
	10.0 	; # 1 A₁
	3.0 	; # 2 A₂
	0.0 	; # 3 B
	0.0 	; # 4 P
	1.0 	; # 5 C
	0.0 	; # 6 x
	0.0 	; # 7 y
];

__Setup constraints__: Next, let's set up the problem's constraints. We'll specify the stoichiometric matrix, the flux bounds array, and species bounds array. 
* The stoichiometric matrix $\mathbf{S}\in\mathbb{R}^{|\mathcal{M}|\times|\mathcal{R}|}$, where $|\mathcal{M}|$ = 7, and $|\mathcal{R}|$ = 3 holds the digital form of the chemical reactions. Species on the rows, reactions on the columns. We store the stoichiometric array in the `S::Array{Float64,2}` variable.
* The flux bounds array hold the permissible ranges of the open extents for each reaction in the system. The flux bounds array will be a $|\mathcal{R}|\times{2}$ array with the reactions on the rows, the lower bound is the first column, and the upper bound is the second. We store the flux bounds in the `flux_bounds_array::Array{Float64,2}` variable.
* The species bounds array holds the information used to compute the right-hand side vector of the constraints. The species bounds array will be a $|\mathcal{M}|\times{2}$ array. We store the species bounds in the `species_bounds_array::Array{Float64,2}` array.

In [13]:
S, flux_bounds_array, species_bounds_array = let

    # Hard code the stoichiometric matrix for the toy example
	S = [
		# r₁ r₂ r₃
		-1.0 0.0 0.0 ; # 1 A₁
		0.0 0.0 -1.0 ; # 2 A₂
		1.0 -1.0 0.0 ; # 3 B
		0.0 1.0 0.0  ; # 4 P
		0.0 0.0 1.0  ; # 5 C
		-1.0 0.0 1.0 ; # 6 x
		1.0 0.0 -1.0 ; # 7 y 
	];

    # set the flux bounds array for the top example (col 1 = lower bound, col 2 = upper bound)
	flux_bounds_array = [

		# ℒ 𝒰
		0.0 10.0 	; # 1 r₁
		0.0 10.0  	; # 2 r₂
		0.0 10.0 	; # 3 r₃
	];

    # set the species bounds array -
	species_bounds_array = [

		# ℒ lower     𝒰 upper
		n_dot_in[1] Inf 				; # 1 A₁
		n_dot_in[2] Inf 				; # 2 A₂
		n_dot_in[3] Inf 				; # 3 B
		n_dot_in[4] Inf 				; # 4 P
		n_dot_in[5] Inf 				; # 5 C
		n_dot_in[6] Inf 				; # 6 x
		n_dot_in[7] Inf 				; # 7 y
	];

    # return -
    S, flux_bounds_array, species_bounds_array
end;

__Objective__: The last setup component in the linear program is the objective coefficients vector. This tells the linear programming problem what we are trying to maximize (or minimize). The objective vector will be $|\mathcal{R}|$-dimensional vector. Positive coefficients maximize, negative coefficients minimize.

In [15]:
objective = [
    0.0  ; # 1 objective function coefficient for r₁
    1.0  ; # 2 objective function coefficient for r₂
    0.0  ; # 3 objective function coefficient for r₃
];

__Setup the model__: To store all the problem data, we created [the `MyOptimalOpenExtentProblemCalculationModel` type](src/Types.jl). Let's build one of these objects for our problem and store it in the `model::MyOptimalOpenExtentProblemCalculationModel` variable. 
* __Builder (or factory) pattern__: For all custom types that we make, we'll use something like [the builder software pattern](https://en.wikipedia.org/wiki/Builder_pattern) to construct and initialize these objects. The calling syntax will be the same for all types: [a `build(...)` method](src/Factory.jl) will take the kind of thing we want to build in the first argument, and the data needed to build that type as [a `NamedTuple` instance](https://docs.julialang.org/en/v1/base/base/#Core.NamedTuple) in the second argument.
* __What's the story with the `let` block__? A [let block](https://docs.julialang.org/en/v1/manual/variables-and-scoping/#Let-Blocks) creates a new hard scope and introduces new variable bindings each time they run. Thus, they act like a private scratch space, where data comes in (is captured by the block), but only what we want to be exposed comes out. 

In [17]:
model = let

    # build -
    model = build(MyOptimalOpenExtentProblemCalculationModel, (
        S = S, # stoichiometric matrix
        fluxbounds = flux_bounds_array, # flux bounds
        speciesbounds = species_bounds_array, # species bounds (needed for right hand side vector b)
        objective = objective, # objective coeffients 
        species = ["A₁", "A₂", "B", "P", "C", "x", "y"], # species (for display purposes later)
        reactions = [
    		"A₁ + x => B + y" 	; # 1 r₁
    		"B => P" 			; # 2 r₂
    		"A₂ + y => C + x" 	; # 3 r₃
    	], # reactions (for display purposes later)
    ));

    # return the model to the caller
    model;
end;

__Compute the optimal open extent__: Finally, let's compute the optimal open extent $\dot{\epsilon}_{r}$ by solving the [linear programming problem](). We solve the optimization problem by passing the `model::MyOptimalOpenExtentProblemCalculationModel` to [the `solve(...)` method](src/Compute.jl). This method returns a `solution::Dict{String, Any}` dictionary, which holds information about the solution.
* Why the [try-catch environment](https://docs.julialang.org/en/v1/base/base/#try)? The [solve(...) method](src/Compute.jl) has an [@assert statement](https://docs.julialang.org/en/v1/base/base/#Base.@assert) to check if the calculation has converged. Thus, the solve method can [throw](https://docs.julialang.org/en/v1/base/base/#Core.throw) an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError) if the optimization problem fails to converge. To gracefully handle this case, we use a [try-catch construct](https://docs.julialang.org/en/v1/base/base/#try). See the [is_solved_and_feasible method from the JuMP package](https://jump.dev/JuMP.jl/stable/api/JuMP/#JuMP.is_solved_and_feasible) for more information.

In [19]:
solution = let

    solution = nothing; # initialize nothing for the solution
    try
        solution = solve(model); # call the solve method with our problem model -
    catch error
        println("error: $(error)"); # Oooooops! Looks like we have a *major malfunction*, problem didn't solve
    end

    # return solution
    solution
end;

In [20]:
solution

Dict{String, Any} with 2 entries:
  "argmax"          => [3.0, 3.0, 3.0]
  "objective_value" => 3.0

__Flux table__: Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal open reaction extents. `Unhide` the code block below to see how we constructed the open extent table.

In [35]:
let

    # setup -
    number_of_reactions = size(S,2); # columns
	flux_table = Array{Any,2}(undef,number_of_reactions,4)
    flux = solution["argmax"];
    
    # populate the state table -
	for reaction_index = 1:number_of_reactions
		flux_table[reaction_index,1] = model.reactions[reaction_index]
		flux_table[reaction_index,2] = flux[reaction_index]
		flux_table[reaction_index,3] = flux_bounds_array[reaction_index,1]
		flux_table[reaction_index,4] = flux_bounds_array[reaction_index,2]
	end

    # header row -
	flux_table_header_row = (["Reaction","ϵᵢ_dot", "ϵ₁_dot LB", "ϵ₁_dot UB"],["","mol/time", "mol/time", "mol/time"]);
		
	# write the table -
	pretty_table(flux_table; header=flux_table_header_row, tf=tf_simple)
end

================== ========== =========== ============
         Reaction     ϵᵢ_dot   ϵ₁_dot LB   ϵ₁_dot UB 
                    mol/time    mol/time    mol/time 
================== ========== =========== ============
  A₁ + x => B + y        3.0         0.0        10.0
           B => P        3.0         0.0        10.0
  A₂ + y => C + x        3.0         0.0        10.0
================== ========== =========== ============


__Stream composition table__: Let's use [the `pretty_tables(...)` method exported by the `PrettyTables.jl` package](https://github.com/ronisbr/PrettyTables.jl) to display the estimated optimal input and output logical stream composition table. `Unhide` the code block below to see how we constructed the composition table.

In [37]:
let

    # get solution and problem components -
    number_of_states = size(S,1); # rows: metabolites
    flux = solution["argmax"];
    n_dot_in = species_bounds_array[:,1];
    
    # compute output -
	ϵ_vector = flux;
	n_dot_out = n_dot_in .+ S*ϵ_vector

	# make a pretty table -
	state_table = Array{Any,2}(undef, number_of_states,3)
	species_array = model.species;
	for state_index = 1:number_of_states
		state_table[state_index,1] = species_array[state_index]
		state_table[state_index,2] = n_dot_in[state_index]
		state_table[state_index,3] = n_dot_out[state_index]
	end

    # header row -
	state_table_header_row = (["Species","nᵢ_dot_in","nᵢ_dot_out"],["","mol/time", "mol/time"]);
		
	# write the table -
	pretty_table(state_table; header=state_table_header_row, tf = tf_simple)
end

LoadError: UndefVarError: `S₁` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

# Today?
That's a wrap! What are some things we discussed today?